In [ ]:
job_worker_name = "k_t_1e5aug_random_building"
!pip uninstall -y fashion_nets
!pip uninstall -y fashion_datasets

In [ ]:
#@title Pip Installs & Imports
#!pip uninstall -y fashion_nets
!pip install -q git+https://github.com/NiklasHoltmeyer/FashionDatasets.git@6843aa034bed6823644557516bbff142d277c6fe
!pip install -q git+https://github.com/NiklasHoltmeyer/FashionNets.git@180ed092e85c02f239dac78a8995337217e6b635

try: 
    from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
    from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job
except:
    from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
    from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job

from fashionnets.train_jobs.loader.dataset_loader import build_dataset_hard_pairs_deep_fashion_1
import tensorflow as tf
from fashionnets.train_jobs.loader.dataset_loader import total_epochs

In [ ]:
#@title Prepare Environment. Download Datasets
from fashionnets.train_jobs.loader.job_loader import add_back_bone_to_train_job

environment, training_job_cfg = prepare_environment(job_worker_name, debugging=False)
import kaggle #<- Requires Secrets, therefore prepare_environment needs to be run before
kaggle.api.authenticate()

kaggle_downloader = kaggle.api.dataset_download_files #<- sadly needs to be injected

train_job = load_job_settings(environment=environment, training_job_cfg=training_job_cfg, kaggle_downloader=kaggle_downloader)
job_settings = add_back_bone_to_train_job(environment=environment, **training_job_cfg)

In [ ]:
siamese_model, init_epoch, _callbacks = load_siamese_model_from_train_job(load_weights=True, **train_job)
print("init_epoch", init_epoch)
print("Callbacks", _callbacks)

In [ ]:
import os 
build_frequency = 1

for i in range(100):
    total_epochs_ = total_epochs(init_epoch, build_frequency)

    dataset = build_dataset_hard_pairs_deep_fashion_1(None, 
                            job_settings, 
                            init_epoch=init_epoch, 
                            build_frequency=build_frequency)   

    history = siamese_model.fit(dataset["train"], 
                            epochs=total_epochs_, 
                            validation_data=dataset["val"],                             
                            initial_epoch=init_epoch, 
                            callbacks = _callbacks)
    
    init_epoch += build_frequency

In [ ]:
_callbacks[-1].on_epoch_end(1000, None)

In [ ]:
!ls ./results